In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 航班数据
fileName = open(r"D:/Python/毕设实验部分/TUYU/data/final_feature.csv")
feature = pd.read_csv(fileName)
feature.head()

,航班编号,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,...,本时段实际进港延误航班数,本时段离港延误时长,本时段离港平均延误时长,本时段离港延误率,上一时段离港平均延误时长,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,标签
0,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,CZ3295,16801,16801,0,1,1.166667,NaN,NaN,0.324175,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [5]:
feature0 = feature[feature['标签'] == 0]
feature0new = feature0.sample(frac=0.15, replace=False, random_state=None,axis=0)
feature1 = feature[feature['标签'] == 1]
featureGBDT = pd.concat([feature0new,feature1],axis=0, ignore_index=True)

In [6]:
del(featureGBDT['本时段离港平均延误时长'])
del(featureGBDT['上一时段离港平均延误时长'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
featureGBDT['起飞间隔'] = featureGBDT['起飞间隔'].fillna(0)
featureGBDT['前序延误'] = featureGBDT['前序延误'].fillna(0)
featureGBDT['出发机场天气得分'] = featureGBDT['出发机场天气得分'].interpolate()
featureGBDT['到达机场天气得分'] = featureGBDT['到达机场天气得分'].interpolate()
featureGBDT['平均延误时间'] = featureGBDT['平均延误时间'].interpolate()
featureGBDT['最大延误时间'] = featureGBDT['最大延误时间'].interpolate()
featureGBDT['延误时间中位数'] = featureGBDT['延误时间中位数'].interpolate()
featureGBDT['延误时间标准差'] = featureGBDT['延误时间标准差'].interpolate()

featureGBDT['本时段计划离港航班数'] = featureGBDT['本时段计划离港航班数'].fillna(featureGBDT['本时段计划离港航班数'].mean())
featureGBDT['本时段实际离港航班数'] = featureGBDT['本时段实际离港航班数'].fillna(featureGBDT['本时段实际离港航班数'].mean())
featureGBDT['本时段进港延误时长'] = featureGBDT['本时段进港延误时长'].fillna(featureGBDT['本时段进港延误时长'].mean())
featureGBDT['本时段实际进港延误航班数'] = featureGBDT['本时段实际进港延误航班数'].fillna(featureGBDT['本时段实际进港延误航班数'].mean())
featureGBDT['本时段离港延误时长'] = featureGBDT['本时段离港延误时长'].fillna(featureGBDT['本时段离港延误时长'].mean())
featureGBDT['本时段离港延误率'] = featureGBDT['本时段离港延误率'].fillna(featureGBDT['本时段离港延误率'].mean())
featureGBDT['上一时段进港延误时长'] = featureGBDT['上一时段进港延误时长'].fillna(featureGBDT['上一时段进港延误时长'].mean())
featureGBDT['上一时段离港延误时长'] = featureGBDT['上一时段离港延误时长'].fillna(featureGBDT['上一时段离港延误时长'].mean())
featureGBDT['上一时段实际进港航班数'] = featureGBDT['上一时段实际进港航班数'].fillna(featureGBDT['上一时段实际进港航班数'].mean())
featureGBDT['上一时段进港延误率'] = featureGBDT['上一时段进港延误率'].fillna(featureGBDT['上一时段进港延误率'].mean())




In [7]:
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

In [8]:
#欠采样下GBDT模型准确率

from sklearn.ensemble import GradientBoostingClassifier   #分类模型
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import numpy as np


dataset=featureGBDT.as_matrix()
X = dataset[:,1:36]
Y = dataset[:,36]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

gbdt = GradientBoostingClassifier(
    loss='deviance',
    learning_rate=0.01, #学习速率
    n_estimators=500,    #训练迭代次数
    subsample=0.6,      #每次训练随机抽取的样本集大小为60%
    max_features= 'sqrt',
    max_depth=6,
    verbose = 2
)

gbdt_model = gbdt.fit(x_train, y_train.astype('int'))  #模型训练开始

#特征值权重排序
#feat_labels = x_train.columns[2:]   #特征列名
# importances = gbdt_model.feature_importances_          #feature_importances_特征列重要性占比
# indices = np.argsort(importances)[::-1]               #对参数从小到大排序的索引序号取逆,即最重要特征索引——>最不重要特征索引
# print(indices)
# print(len(indices))
 
# for f in range(len(indices)):
#     print("%2d) %-*s %f" % (f, 30, indices[f], importances[indices[f]]))

#利用训练的模型来测试  训练集
prediction_train = gbdt_model.predict(x_train)
prediction_train_predprob = gbdt_model.predict_proba(x_train)[:,1]
 
# #利用训练的模型来测试  验证集集
prediction_test = gbdt_model.predict(x_test)
prediction_test_predprob = gbdt_model.predict_proba(x_test)[:,1]



prediction_test = prediction_test.tolist()
y_test = y_test.tolist()
prediction_test = np.array(prediction_test)
y_test = np.array([i for i in y_test])

y_train = y_train.tolist()
y_train = np.array([i for i in y_train])

prediction_train = prediction_train.tolist()
prediction_train = np.array(prediction_train)


print("欠采样下GBDT模型评估")
testAccuracy = metrics.accuracy_score(y_train, prediction_train)
print("训练样本的准确率: %.4f" % testAccuracy)
 
average_precision = metrics.average_precision_score(y_train, prediction_train)
print("average_precision: %.4f" % average_precision)
 
#查准率
averagePrecisionScore = metrics.precision_score(y_train, prediction_train)
print("查准率: %.4f" % averagePrecisionScore)
 
##召回率
returnResultScore = metrics.recall_score(y_train, prediction_train)
print("召回率: %.4f" % returnResultScore)
 
##F1值
F1Score = metrics.f1_score(y_train, prediction_train)
print("F1-Score: %.4f" % F1Score)
 
#计算auc
roc_auc_predprob = metrics.roc_auc_score(y_train, prediction_train_predprob)
print("AUC-Score_predprob: %.4f" % roc_auc_predprob)
 

testAccuracy = metrics.accuracy_score(y_test, prediction_test)
print("验证样本的准确率: %.4f" % testAccuracy)
 
#查准率
averagePrecisionScore = metrics.precision_score(y_test, prediction_test)
print("查准率: %.4f" % averagePrecisionScore)
 
##召回率
returnResultScore = metrics.recall_score(y_test, prediction_test)
print("召回率: %.4f" % returnResultScore)
 
##F1值
F1Score = metrics.f1_score(y_test, prediction_test)
print("F1-Score: %.4f" % F1Score)
 
#计算auc
roc_auc = metrics.roc_auc_score(y_test, prediction_test_predprob)
print("AUC-Score: %.4f" % roc_auc)


ks_score = ks(prediction_test, y_test)
print( ks_score)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.2787           0.0058           13.97m
         2           1.2674           0.0117           14.05m
         3           1.2558           0.0110           13.67m
         4           1.2502           0.0067           13.68m
         5           1.2443           0.0064           13.55m
         6           1.2385           0.0056           13.41m
         7           1.2316           0.0064           13.33m
         8           1.2245           0.0068           13.26m
         9           1.2155           0.0097           13.15m
        10           1.2089           0.0063           13.07m
        11           1.1998           0.0090           13.07m
        12           1.1928           0.0067           13.01m
        13           1.1901           0.0030           12.91m
        14           1.1812           0.0097           12.87m
        15           1.1735           0.0069           12.77m
       

       133           0.7413           0.0008            9.84m
       134           0.7387           0.0022            9.81m
       135           0.7367           0.0013            9.79m
       136           0.7355           0.0022            9.77m
       137           0.7346           0.0014            9.75m
       138           0.7336           0.0010            9.72m
       139           0.7324           0.0019            9.69m
       140           0.7286           0.0018            9.67m
       141           0.7283           0.0016            9.64m
       142           0.7256           0.0020            9.62m
       143           0.7255           0.0013            9.59m
       144           0.7235           0.0011            9.56m
       145           0.7216           0.0020            9.54m
       146           0.7203           0.0017            9.51m
       147           0.7187           0.0013            9.48m
       148           0.7172           0.0010            9.45m
       1

       266           0.6123           0.0004            6.31m
       267           0.6118           0.0009            6.29m
       268           0.6103           0.0005            6.26m
       269           0.6097           0.0002            6.23m
       270           0.6096           0.0004            6.21m
       271           0.6094           0.0005            6.18m
       272           0.6099           0.0004            6.15m
       273           0.6096           0.0003            6.12m
       274           0.6078           0.0010            6.10m
       275           0.6079           0.0009            6.07m
       276           0.6068           0.0005            6.04m
       277           0.6070           0.0007            6.02m
       278           0.6053           0.0005            5.99m
       279           0.6042           0.0004            5.96m
       280           0.6045           0.0004            5.94m
       281           0.6040           0.0008            5.91m
       2

       399           0.5597           0.0004            2.73m
       400           0.5589           0.0002            2.71m
       401           0.5597           0.0003            2.68m
       402           0.5594           0.0003            2.65m
       403           0.5584           0.0003            2.63m
       404           0.5591           0.0005            2.60m
       405           0.5580           0.0001            2.57m
       406           0.5579           0.0001            2.54m
       407           0.5580           0.0002            2.52m
       408           0.5570           0.0002            2.49m
       409           0.5585           0.0004            2.46m
       410           0.5595           0.0002            2.44m
       411           0.5557           0.0003            2.41m
       412           0.5575           0.0002            2.38m
       413           0.5568           0.0003            2.35m
       414           0.5563           0.0001            2.33m
       4

In [9]:
from sklearn.metrics import classification_report 
print("GBDT模型评估")
print(classification_report(prediction_test, y_test))

GBDT模型评估
              precision    recall  f1-score   support

           0       0.95      0.91      0.93    224179
           1       0.81      0.90      0.85     98924

    accuracy                           0.90    323103
   macro avg       0.88      0.90      0.89    323103
weighted avg       0.91      0.90      0.91    323103



In [10]:
import pickle
f= open('D:/Python/毕设实验部分/TUYU/model/model/gbdt_model.pickle','wb')
pickle.dump(gbdt_model,f)
f.close()